In [ ]:
ENV["Code"] = "../../Code"
[push!(LOAD_PATH, string(ENV["Code"],"/", ii)) for ii in readdir(ENV["Code"])]
include("../Code/Create_MF_GeneClass_Table.jl")

In [ ]:
GeneClassTB = CSV.read("tables/MF_GeneClass.csv", DataFrame)

tb0 = dropmissing!(Cuartero2018.GroseqDeseq("KO0__vs__WT0"))
tb1 = dropmissing!(Cuartero2018.GroseqDeseq("KO1__vs__WT1"))
tb6 = dropmissing!(Cuartero2018.GroseqDeseq("KO6__vs__WT6"))

de0 = dropmissing!(Cuartero2018.Cuartero2018Deseq("FLUT_Minus_WTUT"))
de2 = dropmissing!(Cuartero2018.Cuartero2018Deseq("FL2_Minus_WT2"))
de8 = dropmissing!(Cuartero2018.Cuartero2018Deseq("FL8_Minus_WT8"));

In [ ]:
function figure_de_gro_cor_induciblegenes(de, gro, pref1, pref2)
    figure(figsize = (3, 3))

    tb = innerjoin(de, gro, on = :EnsemblID, makeunique = true)
    tb = innerjoin(dropmissing!(GeneClassTB),tb , on = :GeneSymbol)
    tb = tb[tb[!,:Class].=="Inducible", :]
    
    
    tbde1 = []
    tbde2 = []
    for ii in 1:nrow(tb)
        if tb[ii, "padj"] .> 0.05
            push!(tbde1, split(pref1)[1]*":ns")
        elseif tb[ii, "log2FoldChange"] .> 0
            push!(tbde1, split(pref1)[1]*":up")
        elseif tb[ii, "log2FoldChange"] .< 0
            push!(tbde1, split(pref1)[1]*":down")
        end
        if tb[ii, "padj_1"] .> 0.05
            push!(tbde2, split(pref2)[1]*":ns")
        elseif tb[ii, "log2FoldChange_1"] .> 0
            push!(tbde2, split(pref2)[1]*":up")
        elseif tb[ii, "log2FoldChange_1"] .< 0
            push!(tbde2, split(pref2)[1]*":down")
        end
    end
    
    tb[!,"DE1"] = tbde1
    tb[!,"DE2"] = tbde2
    tb[!,"DE"] = tbde1.*" ".*tbde2

    name  = replace(pref1, " "=>"")*"_"*replace(pref2, " "=>"")
    
    tb = tb[tb[!,"DE"].!=split(pref1)[1]*":ns".*" ".*split(pref2)[1]*":ns", :]
    tb = tb[tb[!,"DE1"].!=split(pref1)[1]*":ns", :]
    sort!(tb, "DE", rev = true)
    
    pd = Pandas.DataFrame(tb)
    
    Seaborn.lmplot(data = pd, x = "log2FoldChange", y = "log2FoldChange_1", hue = "DE", fit_reg = false, scatter_kws=Dict("alpha"=>0.5),
        #Matthias colours:
        #RNA-seq up GRO-seq up red, 
        #RNA-seq up GRO-seq NS light grey, RNA-seq up GRO-seq down orange
        #RNA-seq down GRO-seq down blue, 
        #RNA-seq down GRO-seq NS medium grey, 
        #RNA-seq down GRO-seq up turquoise 
        
        
        palette = Dict(
        "RNAseq:up GROseq:up"=> "red",
        "RNAseq:up GROseq:ns"=> "lightgrey",
        "RNAseq:up GROseq:down"=> "orange",
        "RNAseq:down GROseq:down"=> "blue",
        "RNAseq:down GROseq:ns"=> "darkgrey",
        "RNAseq:down GROseq:up"=> "turquoise",
        
        )
    
    )
    
    xlabel("log2FC \n"*pref1)
    
    ylabel("log2FC \n"*pref2)
    
    legend_removal()
    x = tb[!,"log2FoldChange"]; y = tb[!,"log2FoldChange_1"]
    print(R"cor.test($x,$y)")
    
        ax = gca()
ax.set_aspect(1.0/ax.get_data_ratio(), adjustable="box")

    fname = name*"_LPSinducible.svg"
    
    py"""
import matplotlib as mpl

new_rc_params = {'text.usetex': False,
"svg.fonttype": 'none'
}
mpl.rcParams.update(new_rc_params)
import matplotlib.pyplot as plt
plt.savefig($fname)
"""

    test = R"cor.test($x,$y)"
    test[3]
end



figure_de_gro_cor_induciblegenes(de2, tb1, "RNAseq Rad21KO vs WT 2h LPS", "GROseq Rad21KO vs WT 2h LPS")

In [ ]:
import Pkg; Pkg.status()